<a href="https://colab.research.google.com/github/Emanalytics7/LSTM_from_Scratch/blob/main/LSTM__from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM model from scratch using Python


In [15]:
from tqdm import tqdm
import numpy as np

In [16]:
##### Data #####
data = """To be, or not to be, that is the question: Whether \
'tis nobler in the mind to suffer The slings and arrows of ou\
trageous fortune, Or to take arms against a sea of troubles A\
nd by opposing end them. To die—to sleep, No more; and by a s\
leep to say we end The heart-ache and the thousand natural sh\
ocks That flesh is heir to: 'tis a consummation Devoutly to b\
e wish'd. To die, to sleep; To sleep, perchance to dream—ay, \
there's the rub: For in that sleep of death what dreams may c\
ome, When we have shuffled off this mortal coil, Must give us\
 pause—there's the respect That makes calamity of so long lif\
e. For who would bear the whips and scorns of time, Th'oppres\
sor's wrong, the proud man's contumely, The pangs of dispriz'\
d love, the law's delay, The insolence of office, and the spu\
rns That patient merit of th'unworthy takes, When he himself \
might his quietus make""".lower()
chars = set(data)

In [17]:
data_size, char_size = len(data), len(chars)
print(data_size, char_size)

866 32


In [24]:
char_to_idx = {c:i for i, c in enumerate(chars)}
idx_to_char = {i:c for i, c in enumerate(chars)}

char_to_idx

{'d': 0,
 'n': 1,
 'q': 2,
 ':': 3,
 '—': 4,
 't': 5,
 'k': 6,
 'c': 7,
 "'": 8,
 'z': 9,
 ',': 10,
 'v': 11,
 's': 12,
 'a': 13,
 'm': 14,
 'h': 15,
 'o': 16,
 'e': 17,
 'b': 18,
 'p': 19,
 '.': 20,
 'f': 21,
 'i': 22,
 'w': 23,
 '-': 24,
 'u': 25,
 'r': 26,
 ';': 27,
 ' ': 28,
 'g': 29,
 'y': 30,
 'l': 31}

In [25]:
train_x, train_y = data[: -1], data[1:]

#### Uniform Xavier Initialization  
draw each weight, w, from a random uniform distribution in in [-x,x] for

$x = \sqrt{\frac{6}{\text{inputs} + \text{outputs}}}$

In [26]:
def oneHotEncode(text):
    output = np.zeros((char_size, 1)) # 32 rows with 1 column
    output[char_to_idx[text]] = 1
    return output

def initWeights(input_size,  output_size):
    return np.random.uniform(-1, 1, (output_size, input_size))  * np.sqrt(6 / (input_size + output_size))

In [27]:
# activation function

def sigmoid(input, derivative=False):
    if derivative:
        return input * (1 - input)

    return 1 / (1 + np.exp(-input))


def tanh(input, derivative = False):
    if derivative:
        return 1 - input ** 2
    return np.tanh(input)

def softmax(input):
     return np.exp(input) / np.sum(np.exp(input))

### Tanh Derivative Definition  
$$  
\frac{d}{dx} \tanh(x) = 1 - \tanh^2(x)  
$$  

### Sigmoid Derivative Definition  
$$  
\frac{d}{dx} \sigma(x) = \sigma(x)(1 - \sigma(x))  
$$

In [30]:
## Long short-term memory network class
class LSTM:
    def __init__(self, input_size, hidden_size, output_size, num_epochs, learning_rate):

         ## hyperparameters
         self.learning_rate = learning_rate
         self.hidden_size = hidden_size
         self. num_epochs = num_epochs

        ##### weight and baises for the gates

         ## forget gate
         self.wf = initWeights(input_size, hidden_size)
         self.bf = np.zeros((hidden_size, 1))

         ## input gate
         self.wi = initWeights(input_size, hidden_size)
         self.bi = np.zeros((hidden_size, 1))

         ## candidate gate
         self.wc = initWeights(input_size, hidden_size)
         self.bc = np.zeros((hidden_size, 1))

         ## output gate
         self.wo = initWeights(input_size, hidden_size)
         self.bo = np.zeros((hidden_size, 1))

         ## final gate
         self.wy = initWeights(hidden_size, output_size)
         self.by = np.zeros((output_size, 1))

    def reset(self):
        self.concat_inputs = {}

        self.hidden_states = {-1:np.zeros((self.hidden_size, 1))}
        self.cell_states = {-1:np.zeros((self.hidden_size, 1))}

        self.activation_outputs = {}
        self.candidate_gates = {}
        self.forget_gates = {}
        self.output_gates = {}
        self.input_gates = {}
        self.outputs = {}

    def forward(self, inputs):
        self.reset()

        outputs = []
        for r in range(len(inputs)):
            self.concat_inputs[r] = np.vstack((self.hidden_states[r - 1], inputs[r]))

            self.forget_gates[r] = sigmoid(np.dot(self.wf, self.concat_inputs[r]) + self.bf)
            self.input_gates[r] = sigmoid(np.dot(self.wi, self.concat_inputs[r]) + self.bi)
            self.candidate_gates[r] = tanh(np.dot(self.wc, self.concat_inputs[r]) + self.bc)
            self.output_gates[r] = sigmoid(np.dot(self.wo, self.concat_inputs[r]) + self.bo)

            self.cell_states[r] = self.forget_gates[r] * self.cell_states[r - 1] + self.input_gates[r] * self.candidate_gates[r]
            self.hidden_states[r] = self.output_gates[r] * tanh(self.cell_states[r])

            outputs += [np.dot(self.wy, self.hidden_states[r]) + self.by]

        return outputs


    def backward(self, errors, inputs):
        d_wf, d_bf = 0, 0
        d_wi, d_bi = 0, 0
        d_wc, d_bc = 0, 0
        d_wo, d_bo = 0, 0
        d_wy, d_by = 0, 0

        dh_next, dc_next = np.zeros_like(self.hidden_states[0]), np.zeros_like(self.cell_states[0])
        for r in reversed(range(len(inputs))):
            error = errors[r]

            ## final gate weights and biases errors
            d_wy += np.dot(error, self.hidden_states[r].T)
            d_by += error

            """for each time step `r`, calculated the gradients for the output
             weights d_wy and baises d_by

                -> d_wy += np.dot([0.01], [h2].T) hidden state from time step 2
            """


            ## hidden state error

            d_hs = np.dot(self.wy.T, error) + dh_next

            """ calculating the propogated error to the hidden state
            """
            ## output gate weights and biases errors

            d_o = tanh(self.cell_states[r]) * d_hs * sigmoid(self.output_gates[r], derivative=True)
            d_wo += np.dot(d_o, inputs[r].T)
            d_bo += d_o


            # cell state error
            d_cs = tanh(tanh(self.cell_states[r], derivative=True) * self.output_gates[r] * d_hs + dc_next)

            # forget gate weights and biases errors
            d_f = d_cs * self.cell_states[r - 1] * sigmoid(self.forget_gates[r], derivative=True)
            d_wf += np.dot(d_f, inputs[r].T)
            d_bf += d_f

            ## input gate weights and biases error
                        # Input Gate Weights and Biases Errors
            d_i = d_cs * self.candidate_gates[r] * sigmoid(self.input_gates[r], derivative = True)
            d_wi += np.dot(d_i, inputs[r].T)
            d_bi += d_i

            ## candidate Gate Weights and Biases Errors
            d_c = d_cs * self.input_gates[r] * tanh(self.candidate_gates[r], derivative = True)
            d_wc += np.dot(d_c, inputs[r].T)
            d_bc += d_c

            ## concatenated Input Error (Sum of Error at Each Gate!)
            d_z = np.dot(self.wf.T, d_f) + np.dot(self.wi.T, d_i) + np.dot(self.wc.T, d_c) + np.dot(self.wo.T, d_o)

            ## error of Hidden State and Cell State at Next Time Step
            dh_next = d_z[:self.hidden_size, :]
            dc_next = self.forget_gates[r] * d_cs

        for d_ in (d_wf, d_bf, d_wi, d_bi, d_wc, d_bc, d_wo, d_bo, d_wy, d_by):
            np.clip(d_, -1, 1, out = d_)

        self.wf += d_wf * self.learning_rate
        self.bf += d_bf * self.learning_rate

        self.wi += d_wi * self.learning_rate
        self.bi += d_bi * self.learning_rate

        self.wc += d_wc * self.learning_rate
        self.bc += d_bc * self.learning_rate

        self.wo += d_wo * self.learning_rate
        self.bo += d_bo * self.learning_rate

        self.wy += d_wy * self.learning_rate
        self.by += d_by * self.learning_rate


    def train(self, inputs, labels):
        inputs = [oneHotEncode(input) for input in inputs]

        for _ in tqdm(range(self.num_epochs)):
            predictions = self.forward(inputs)

            errors = []
            for r in range(len(predictions)):
                errors += [-softmax(predictions[r])]
                errors[-1][char_to_idx[labels[r]]] += 1
            self.backward(errors, self.concat_inputs)

    def test(self, inputs, labels):
        accuracy = 0
        probabilities = self.forward([oneHotEncode(input) for input in inputs])

        output = ''
        for  r in range(len(labels)):
            predictions = idx_to_char[np.random.choice([*range(char_size)], p=softmax(probabilities[r].reshape(-1)))]
            output += predictions

            if predictions == labels[r]:
                accuracy += 1

        print(f'Ground Truth:\nt{labels}\n')
        print(f'Predictions:\nt{"".join(output)}\n')
        print(f'Accuracy : {round(accuracy*100/len(inputs), 2)}%')


In [31]:
hidden_size = 25

lstm  = LSTM(input_size = char_size + hidden_size, hidden_size=hidden_size, output_size=char_size, num_epochs=1_000, learning_rate=0.05)
lstm.train(train_x, train_y)
lstm.test(train_x, train_y)

100%|██████████| 1000/1000 [02:12<00:00,  7.53it/s]


Ground Truth:
to be, or not to be, that is the question: whether 'tis nobler in the mind to suffer the slings and arrows of outrageous fortune, or to take arms against a sea of troubles and by opposing end them. to die—to sleep, no more; and by a sleep to say we end the heart-ache and the thousand natural shocks that flesh is heir to: 'tis a consummation devoutly to be wish'd. to die, to sleep; to sleep, perchance to dream—ay, there's the rub: for in that sleep of death what dreams may come, when we have shuffled off this mortal coil, must give us pause—there's the respect that makes calamity of so long life. for who would bear the whips and scorns of time, th'oppressor's wrong, the proud man's contumely, the pangs of dispriz'd love, the law's delay, the insolence of office, and the spurns that patient merit of th'unworthy takes, when he himself might his quietus make

Predictions:
to bg  or tat 'l by, nhat az 'ha suestoon: wea,lepestimhaos' r onewoarhang tormlrfirctoe phiel  oni orosc

Of course, it’s not going to provide 99.9% accuracy, but it’s just for learning purposes! :)